# Life 101

In this notebook, we create a simple analogy between Wikispeedia game and "Life". We found out that some pieces of advice extracted from the game are also applicable in real life and we can learn from them!

# Data Preparation

Please add folders `wikispeedia_articles_html`, `wikispeedia_articles_plaintext`, `wikispeedia_paths-and-graph` from the orignal dataset.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import src.utils as utils
import plotly.express as px
import plotly.graph_objects as go



# Automatically reload the module in case of changes
%load_ext autoreload
%autoreload 2

In [2]:
data_path = "./data/wikispeedia_paths-and-graph/"

# Load finished paths
finished_paths = utils.load_dataframe(data_path + "paths_finished.tsv", columns=["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"])

# Load unfinished paths
unfinished_paths = utils.load_dataframe(data_path + "paths_unfinished.tsv", columns=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"])

# Load categories
categories = utils.load_dataframe(data_path + "categories.tsv", columns=["page", "category"])

# Load links
links_df = utils.load_dataframe(data_path + "links.tsv", columns=["source", "target"])

In [4]:
# Data cleaning and preprocessing
categories_dict = utils.manage_categories(categories.copy())
finished_paths_df = utils.manage_paths(finished_paths.copy(), categories_dict.copy())
unfinished_paths_df = utils.manage_paths(unfinished_paths.copy(), categories_dict.copy())
links_dict = utils.manage_links(links_df.copy())

#create graph and page ranks
graph = utils.create_graph(links_df.copy())
ranks = utils.page_rank(graph)

# Save the cleaned dataframes
finished_paths_df.to_csv("./data/clean_data/clean_finished_paths.csv")
unfinished_paths_df.to_csv("./data/clean_data/clean_unfinished_paths.csv")

# 1. Don't trust LLMs blindly!

We will compare people performance to LLM performance by considering the most taken paths by people and launching them on a 4bit quantized version of Qwen-3b. A fairly small LLM that could run on our machines/Colab in a reasonable time. We run the same task from source to target 30 times using the LLM. This allows us to have statistically relevant paths especially that the LLM is sometimes lost and not able to finish the paths everytime. The file that generates the LLM results is /utils/LLM.py. Please note that CUDA is required for this file to run. (It may not generate the exact same paths but they will still have the same behaviour statistically.)

## Prompting strategy
In order to get the model to work, we did prompt engineering trying multiple instructions for the model. The overall idea of the prompt is that we give the model only the target and the list of possible links at a certain step and we ask it about the best choice in order to get closer to the target. If the answer is not in the list, we keep the context of the conversation and we tell the model that it gave us an article not in the list and we ask it to correct itself. If it does, then we continue, otherwise, the path is considered aborted.
If we have an answer, we restart with a completely new context using a similar prompt.

## Comparison strategy
As we get multiple samples of answers (30 in our case) from the LLM for each source-target pair, we get an estimate of the distribution of lengths of the paths given by the LLM. Similarly, we have multiple samples of player paths, so we cannot only compare the means of the path lengths but also do a t-test similarity test to know if the distributions are similar or not.

In [5]:
# Group the finished paths DataFrame by "source" and "target" columns, then count occurrences in each group
top_paths = finished_paths_df.groupby(["source", "target"])["hashedIpAddress"].count().rename("count")

# Sort the grouped data by the count in descending order to get the most frequent paths
top_paths = top_paths.sort_values(ascending=False).reset_index().head(10)
print("Top 10 paths: ")
print(top_paths)

# Extract the "source" column values of the top paths as a list
sources = top_paths['source'].tolist()

# Extract the "target" column values of the top paths as a list
targets = top_paths['target'].tolist()

# Get the LLM paths
# uncomment the following lines to run the LLM generation (CUDA is required) and comment the next one
# import src.LLM as LLM
# llm_paths = LLM.llm_paths(sources, targets, links_dict)
llm_paths = utils.read_llm_paths("./data/llm_paths_with_new_prompt2.json")

Top 10 paths: 
     source             target  count
0  Asteroid             Viking   1043
1     Brain          Telephone   1040
2   Theatre              Zebra    905
3   Pyramid               Bean    642
4    Batman               Wood    148
5      Bird  Great_white_shark    138
6    Batman      The_Holocaust    119
7      Bird       Adolf_Hitler    107
8      Beer                Sun     99
9    Batman             Banana     69


In [6]:
# Plot LLM and players mean lengths and errors
utils.plot_llms_vs_players(sources, targets, finished_paths_df, llm_paths)
p_values = utils.tstats_pvalues(sources, targets, finished_paths_df, llm_paths)
utils.plot_tsatistics(sources, targets, p_values)

## Statistical analysis

The first plot shows that the LLM's performance is worse in almost all the source-target pairs. However, we are not confident that it's better in all of them. (For example, the players mean falls into the standard error of the LLM mean for the Brain-Telephoneand Batman-Banana paths). This is why we also calculate the t-test p-values. We define our null hypothesis as "LLMs have larger average path length than human players". We can see that for almost all of the source-target pairs, the p-values are much larger than 0.05 so we cannot reject the null hypothesis. As the p-values are very high, it indicates that our observation are not so unlikely to have occurred by chance. Except for Pyramid-Bean pair for which the p-value is less than 0.05

Not only LLMs are not as good as humans but also they tried to cheat in the game. In fact, when we look at the number of games finished by the LLMs, we can see that for the paths they struggled on, they finished less games than for the easy paths. (example: the 3 source-target pairs with the highest mean lengths are : Bird -> Great white Shark, Theatre -> Zebra and Pyramid -> Bean they are also the 3 source-target pairs with the lowest number of finished paths.)
This means that they twice tried to give us an answer that is not in their list of possibilities so they did not follow the task.

In [7]:
number_of_finished_paths = {
    "Source_Target": [f"{source} -> {target}" for source, target in zip(sources, targets)],
    "Number_of_LLM_paths": [len(llm_paths[f"{source}_{target}"]) for source, target in zip(sources, targets)]
}
fig = px.bar(number_of_finished_paths, x="Source_Target", y="Number_of_LLM_paths", 
             title="Number of LLM paths finished out of 30 for each source-target pair")

fig.update_layout(
    xaxis_title="Source -> Target",
    yaxis_title="Number of LLM paths",
    xaxis_tickangle=45
)
fig.show()

## What is the LLM's strategy?
We studied the LLM strategy to know why they don't get as good performance as real players. We ran page rank algorithm on the graph of links and plotted the average ranks of nodes for the most common path lengths for players and LLMs on each source-target pair. We can see that players tend to go up quickly to a "Hub" and then go down to the target. The LLMs on the other hand don't have a clear pattern in their paths. This confirms to us that they are not good on all tasks and also that they are not very consistent. However, we observed that when they use the "Hub" strategy they are as god as humans (example: Batman -> The Holocaust path).

In [9]:
utils.plot_llm_vs_players_strategies(sources, targets, finished_paths_df, llm_paths, ranks)

## Take away about LLMs
LLMs are a very good tool that could make our lives easier in many ways. However, one should not trust them blindly. They can give us inaccurate or even wrong answers. We can ask them for help and advice but they are still far from human strategic thinking and way of reasoning.